## House Prices: Advanced Regression Techniques

#### Competition Description

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.


### Goal 

To predict the sales price for each house. For each ID in the test set, you must predict the value of the SalePrice variable

### Metric

Submission are evaluated on the Root-Mean-Squared-Error(RMSE) between the logarithm of the predicted value and logarithm of the observed sales price. 
**(Taking logs means that errors in predicting the expensive houses and cheap houses will effect the result equally)**

#### Notes from description
- YearRmodeladd is the same year if there have been no remodel or additions (need to change this to 0)
- There are Total Rooms and Bedrooms, don't confuse them

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm, skew
import datetime as dt
import math
from math import radians, cos, sin, asin,sqrt
import glob
import os
import pandas as pd
import pandas_profiling
pd.set_option('display.max_columns', None)
# Visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import seaborn as sns
#import plotly.plotly as py
import plotly.graph_objs as go
#import plotly
#plotly.tools.set_credentials_file(username='peanuttbuddha', api_key='NJTdnmJo7EwDcaxEL9mO')
import plotly.offline as offline
offline.init_notebook_mode()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chartify
# NOTE THAT INLINE NEEDS TO BE LAST
%matplotlib inline
# Missing Data Visualization
import missingno as msno

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.columns

In [ ]:
train.head()

### Because of the plots below I will get rid of any values where sqft > 4000 and sale price < 500k (just those 2 datapoints)

In [ ]:
train = train.drop(train[((train['GrLivArea']>4000) & (train['SalePrice']<300000))].index)
train.reset_index(drop=True)

In [ ]:
data = train.append(test, ignore_index=True)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
#pandas_profiling.ProfileReport(data)

### That is a lot of information. Let's start with finding and filling missing values and then run the report again
- Alley has 2721 / 93.2% missing values Missing
- BsmtCond has 82 / 2.8% missing values Missing
- BsmtExposure has 82 / 2.8% missing values Missing
- BsmtFinType1 has 79 / 2.7% missing values Missing
- BsmtFinType2 has 80 / 2.7% missing values Missing
- BsmtUnfSF 1 missing values Missing
- BsmtFinSF1 1 missing values Missing
- BsmtFinSF2 1 missing values Missing
- BsmtQual has 81 / 2.8% missing values Missing
- Fence has 2348 / 80.4% missing values Missing
- FireplaceQu has 1420 / 48.6% missing values Missing
- GarageCond has 159 / 5.4% missing values Missing
- GarageFinish has 159 / 5.4% missing values Missing
- GarageQual has 159 / 5.4% missing values Missing
- GarageType has 157 / 5.4% missing values Missing
- GarageYrBlt has 159 / 5.4% missing values Missing
- LotFrontage has 486 / 16.6% missing values Missing
- MiscFeature has 2814 / 96.4% missing values Missing
- PoolQC has 2909 / 99.7% missing values Missing
- SalePrice has 1459 / 50.0% missing values Missing
<br>

**Breaking Down null values** <br>
1. null values where we know we can just impute 'None'
    - Alley we can impute NaN with None, since we can assume they don't have an alley
    - Fence we can also do the same
    - MiscFeature we can also do the same
    - FireplaceQu we can also do the same since we can see that the 'Fireplaces' column has 1420 zeros and FireplaceQu has 1420 NaN
2. Null Values where we need to do more investigation
    - **PoolQC.** We have 13 non 0 values in the Pool Area and only 10 values for PoolQC, meaning there are 3 PoolQC values that are not 'None', so first we need to find the rows that have PoolArea filled in where PoolQC are null and fill those in somehow, then we can impute the rest with 'None'
    - **Bsmt stuff**
    - **Garage stuff**
    - **LotFrontage** Possibly bin LotArea and take the average based on that
    - **Function**
    - **KitchenQual**

**Note that I removed a lot of the code that where I imputed values because it is mostly the same for each feature that I changed**

#### Starting with filling in PoolQC values

In [ ]:
data.loc[(data.PoolArea!=0) & (data.PoolQC.isnull())]

In [ ]:
# Make some graphs to see if I can impute these by that information
fig= plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(121)
sns.boxplot(x='PoolQC', y='PoolArea', data=data, ax=ax1)
ax2 = fig.add_subplot(122)
sns.boxplot(x='PoolQC', y='SalePrice', data=data, ax=ax2)


### There aren't a lot of values in this category, so It doesn't matter too much but based on these graphs I'll fill the smaller pool area values with 'Ex' and the 561 pool area with 'Fa'

In [ ]:
# Changing the specific null values to 'Ex' or 'Fa'
data['PoolQC'].loc[(data['PoolQC'].isnull()) & (data['PoolArea'] < 500) & (data['PoolArea'] !=0)] = 'Ex'
data['PoolQC'].loc[(data['PoolQC'].isnull()) & (data['PoolArea'] > 500)] = 'Fa'

In [ ]:
# checking to make sure we are ready to fill in na values with 0 or none
updated_list_of_na_columns = data.columns[data.isna().any()].tolist()
updated_list_of_na_columns

## All Data is ready for the mass fill besides LotFrontage

In [ ]:
# First need to pull out SalePrice 
lot_frontage = data['LotFrontage']
del data['LotFrontage']

### Mass filling in of null Values

In [ ]:
# Want to drop saleprice first so we can keep those as nans
target= data['SalePrice']
del data['SalePrice']

In [ ]:
# List Comprehension to fill in the null values, if column is Object fillna with None, else fillna with 0
# assigning it to a random variable so 'none' doesn't get printed a million times
_ = [data[col].fillna('None', inplace=True) if (data[col].dtype=='O') else data[col].fillna(0, inplace=True) for col in data]

In [ ]:
# and add back in so we can explore the data
data=data.join(target)

### Data Exploration
- Before I impute Lot frontage with ML I want to explore the data a little
- In this case I don't want to take dummies on objects yet because I want to explore some of the categorical variables as well

#### Since There are a lot of things to compare I will create a function for simple plotly graphs so I can call it whenever

In [ ]:
# Need to do try and except and also multiple kwargs
def plotly_plot(df, colx, coly, chart_type,**kwargs):
    #try:
        #print (go.chart_type)
        trace = chart_type(x=df[colx], y=df[coly], **kwargs)
        plot = [trace]
        layout = go.Layout(
                xaxis=dict(
                    title = colx,
                        titlefont=dict(
                            family='Courier New, monospace',
                                size=18,
                                    color='#000000'
                        )
                ),
                yaxis = dict(
                    title=coly,
                        titlefont=dict(
                            family='Courier New, monospace',
                                size=18,
                                    color='#000000'
                        )
                )
        )
        fig=dict(data=plot,layout=layout)
        return offline.iplot(fig)
    #except:
        #print('Please use (go.) before your chart_type of choice')
    

In [ ]:
plotly_plot(data, 'OverallCond', 'SalePrice', go.Box)

### Its really interesting to see the outliers at the 5 and 6 overallcond. Higher OverallCond does not mean higher price
#### Lets check OverallQual

In [ ]:
plotly_plot(data, 'OverallQual', 'SalePrice', go.Box)

### So here we see the trend 

In [ ]:
## Checking to see if there is a relationship between bedrooms(above ground) and sale price
plotly_plot(data, 'BedroomAbvGr', 'SalePrice', go.Box)

In [ ]:
# what about Total Rooms?
plotly_plot(data, 'TotRmsAbvGrd', 'SalePrice', go.Box)

#### somewhat of a Trend. This makes me think about rooms and sqft.
- Id assume there is a relationship

In [ ]:
plotly_plot(data, 'TotRmsAbvGrd', 'GrLivArea', go.Box)

In [ ]:
bed_bath_group = data.groupby('BedroomAbvGr', as_index=False)['FullBath'].agg('mean')
plotly_plot(bed_bath_group, 'BedroomAbvGr', 'FullBath',go.Scatter)

In [ ]:
# setting up groupbys for the chart
max_bath_group = data.groupby('BedroomAbvGr', as_index=False)['FullBath'].agg('max')
avg_bath_group = data.groupby('BedroomAbvGr', as_index=False)['FullBath'].agg('mean')
min_bath_group = data.groupby('BedroomAbvGr', as_index=False)['FullBath'].agg('min')
# Create and style traces
trace0 = go.Scatter(
    x = max_bath_group['BedroomAbvGr'],
    y = max_bath_group['FullBath'],
    name = 'Max baths',
    line = dict(
        color = ('rgb(76, 153, 0)'),
        width = 4)
)
trace1 = go.Scatter(
    x = avg_bath_group['BedroomAbvGr'],
    y = avg_bath_group['FullBath'],
    name = 'Avg Baths',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,)
)
trace2 = go.Scatter(
    x = min_bath_group['BedroomAbvGr'],
    y = min_bath_group['FullBath'],
    name = 'Min baths',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
        dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)

plot = [trace0, trace1, trace2]

# Edit the layout
layout = dict(title = 'Min, Avg, and Max bathrooms per bedrooms',
              xaxis = dict(title = 'Bedrooms(Above Ground)'),
              yaxis = dict(title = 'Baths'),
              )

fig = dict(data=plot, layout=layout)
iplot(fig)

#### Interesting that some places dont have bathrooms?

In [ ]:
data.loc[(data.FullBath ==0)]

In [ ]:
data.loc[(data.FullBath==0) & (data.BsmtFullBath ==0)]

### These residences do have bathrooms, theyre just in the basement. I can assume these are solid data points
#### Even the one above has some half baths(I'm assuming it could be 3/4 baths?)

### 0 Bedrooms?

In [ ]:
data.loc[(data.BedroomAbvGr==0)]

### They all have fairly large FINISHED basements. This is acceptable

In [ ]:
plotly_plot(data, 'FullBath', 'GrLivArea', go.Box)

#### Possible Outliers? 

In [ ]:
plotly_plot(data, 'GrLivArea', 'SalePrice',go.Scatter, mode='markers')

### Clearly a trend here. 
- Are there outliers we need to get rid of?
    - I think we should get rid of any sqft > 4000 and any sale price > 500k
    - **We need to do this on the train set**! ^ up above I will do it
#### For fun I'm going to add in LotArea and do a 3d

In [ ]:
plotly_plot(data, 'GrLivArea', 'LotArea', go.Scatter3d, mode='markers', z=data['SalePrice'])

In [ ]:
qual_sf_group = data.groupby('OverallQual', as_index=False )['GrLivArea'].agg('mean')
plotly_plot(qual_sf_group, 'OverallQual', 'GrLivArea', go.Bar)

In [ ]:
# Curious if there are any houses where the basement is larger than the 1stFloorSF, that would be weird?
plotly_plot(data, '1stFlrSF', 'TotalBsmtSF', go.Scatter, mode='markers')

### There are some, but theres a pretty linear relationship like I thought

In [ ]:
# groupby to get average
month_sold_group = data.groupby('MoSold', as_index=False)['SalePrice'].agg('mean')
plotly_plot(month_sold_group, 'MoSold', 'SalePrice', go.Scatter, mode='lines')

### Interesting, There is no real correlation between sale price and month, this also shows us that we need to get dummies on 'MoSold'
#### Going to add in min and max per month and see what that looks like

In [ ]:
# setting up groupbys for the chart
max_sold_group = data.groupby('MoSold', as_index=False)['SalePrice'].agg('max')
avg_sold_group = data.groupby('MoSold', as_index=False)['SalePrice'].agg('mean')
min_sold_group = data.groupby('MoSold', as_index=False)['SalePrice'].agg('min')
# Create and style traces
trace0 = go.Scatter(
    x = max_sold_group['MoSold'],
    y = max_sold_group['SalePrice'],
    name = 'Max Sale Price',
    line = dict(
        color = ('rgb(76, 153, 0)'),
        width = 4)
)
trace1 = go.Scatter(
    x = avg_sold_group['MoSold'],
    y = avg_sold_group['SalePrice'],
    name = 'Avg Sale Price',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,)
)
trace2 = go.Scatter(
    x = min_sold_group['MoSold'],
    y = min_sold_group['SalePrice'],
    name = 'Min Sale Price',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
        dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)

plot = [trace0, trace1, trace2]

# Edit the layout
layout = dict(title = 'Min, Avg, and Max Sale Prices per month',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Sale Price'),
              )

fig = dict(data=plot, layout=layout)
iplot(fig)

#### Interesting to see the variance in the max values, but mostly the average doesn't vary too much. Further proof we need to convert MoSold to a string and then take dummies

In [ ]:
plotly_plot(data, 'YrSold', 'SalePrice', go.Box)

### So Strange that YrSold does not effect sale price at all
- Should change this to Categorical and take dummies!

In [ ]:
plotly_plot(data, 'Neighborhood', 'SalePrice', go.Box)

### Feature Engineering
1. replacing year with 0 if yearremodadd=yrbuilt(should not have a value if never remodeled
2. changing numeric col to str(to take dummies)
3. TotalSF = GrlivArea + TotalBsmtSF
4. GrLivArea/ LotArea
5. Lot frontage/lot area


#### 1. If yearremodadd = yearbuilt, replace value in yearremodadd with 0(according to docs this means there was no remodel)
- Surprisingly this did not really do anythin

In [ ]:
# if year built and remodadd are the same replace yearremodadd with 0. They should not have a value if theyve never been remodeled
data['YearRemodAdd']= np.where(data.YearRemodAdd == data.YearBuilt, 0, data.YearRemodAdd) 

### 2. Changing stuff to strings

In [ ]:
# Change MoSold from int to a String so you can take dummies(based on chart above)
def turn_obj(cols):
    for col in cols:
        data[col] = data[col].astype(str)

In [ ]:
turn_obj(['MoSold', 'YrSold', 'OverallCond', 'MSSubClass', 'GarageCars'])

### 3. Total Sq ft(including basement)

In [ ]:
data['TotalSF'] = data['GrLivArea'] + data['TotalBsmtSF']

### 4. Sq ft divided by Lot Area

In [ ]:
data['totalSF_by_LotArea'] = data['TotalSF'] / data['LotArea']

### Remove ID!

In [ ]:
# Need to drop SalePrice because it has NaNs and we dont want it in the algo to impute LotFrontage
# will create target again, just because
target = data['SalePrice']
missing_sales = data[data['SalePrice'].isnull()]
sub_id = missing_sales['Id']
# delete so its not used
#del data['Id']
data.drop(['SalePrice', 'Id'], axis=1,inplace=True)
# Bring in LotFrontage
data=data.join(lot_frontage)

In [ ]:
data.groupby('Neighborhood')['LotFrontage'].agg('mean')

### Try mean and median, see what works best

In [ ]:
# Will first impute missing lotfrontage values with the mean based on the neighborhood
data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# 1 more piece of feature engineering
data['lotarea-frontage'] = data['LotFrontage'] / data['LotArea']

### Is anything too correlated with each other?

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(data.corr(), annot=True)

#### Using Labelencoding, as it has worked for mls data

In [ ]:
from sklearn import preprocessing
cols_for_label = ['BsmtCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','Condition1', 
                  'Condition2', 'ExterQual', 'GarageCond', 'GarageQual', 'GarageType', 'KitchenQual', 'LotShape', 
                 'LotConfig', 'MiscFeature', 'PavedDrive', 'Functional', 'Fence', 'Alley', 'YearRemodAdd']
# loop to use labelencoder on the chosen columns
le = preprocessing.LabelEncoder()
for col in cols_for_label:
    le.fit(data[col])
    list(le.classes_)
    data[col] = le.transform(data[col])


#### Now to check distribution/ skewness

In [ ]:
target_no_nan = target.dropna()

In [ ]:
# this code taken from another user here on kaggle. Great stuff thank you!
def check_skewness(df):
    sns.distplot(df, fit = norm);
    fig =plt.figure(figsize=(16,8))
    res = stats.probplot(df, plot=plt)
    # get fitted parameters used by the function
    (avg, std) = norm.fit(df)
    print ('\n avg = {:.2f} and std = {:.2f}\n' .format(avg, std))
check_skewness(target_no_nan)

In [ ]:
target_no_nan = np.log1p(target_no_nan)

check_skewness(target_no_nan)

### Good Stuff

### Looking at Skew For Features

In [ ]:
num_feats = data.dtypes[data.dtypes != 'object'].index
#check skew
skewed_feats = data[num_feats].apply(lambda x:skew(x)).sort_values(ascending=False)
skewness = pd.DataFrame({'sKew':skewed_feats})
#skewness = skewness.drop(['price'])
skewness.head()

### Fixing Skew For Features

In [ ]:
# Boxcox fix skew
skewness = skewness[abs(skewness) > .75]
print (skewness.shape[0])
from scipy.special import boxcox1p
skewed_features = skewness.index
lam = .15
for feat in skewed_features:
    data[feat] = boxcox1p(data[feat], lam)

In [ ]:
data = data.join(target_no_nan)

#### Get Dummies

In [ ]:
def get_dummies(df):
    future_drop = [col for col in df if df[col].dtype == 'O']
    # I know get dummies only takes Objects but if I don't do the list comp inside it gives me a columns overlap error
    df = df.join(pd.get_dummies(df[[col for col in df if df[col].dtype == 'O']], drop_first=True)).drop(future_drop, axis=1) 
    return df
    #df.drop(future_drop, axis=1, inplace=True)
#data = get_dummies(data)

In [ ]:
data=get_dummies(data)

## Modeling
- Will try a couple models and then average them

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC, LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [ ]:
missing_price = data[data['SalePrice'].isnull()]
filled_price = data[data['SalePrice'].notnull()]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(filled_price.drop('SalePrice', axis=1),filled_price['SalePrice'], test_size=.2, random_state=42)

In [ ]:
# StandardScaler was almost identical to robust but gave warnings
# FunctionTransformer helped but not too noticeable
# RobustScaler worked the best
gbr=  make_pipeline(RobustScaler(),GradientBoostingRegressor(n_estimators=800, learning_rate=0.05,
                                  max_depth=4, max_features='log2',
                                  min_samples_leaf=8, min_samples_split=6,
                                  loss='huber', random_state=42))
br = make_pipeline(RobustScaler(),BayesianRidge())
r = make_pipeline(RobustScaler(),Ridge())
xgb = make_pipeline(RobustScaler(),XGBRegressor())
svr =make_pipeline(RobustScaler(),SVR(kernel='linear'))
# Lasso and enet were way off until I messed with alpha,possibly fine tuning will bring a better scores
l = make_pipeline(RobustScaler(),Lasso(alpha=.0005))
enet = make_pipeline(RobustScaler(), ElasticNet(alpha=.001))

In [ ]:
# going to put cross_val in the tdmassess
def cv_score(algo):
    rmse= np.sqrt(-cross_val_score(algo, X_train, y_train, scoring='neg_mean_squared_error', cv=5))
    return (rmse.mean())

In [ ]:
algorithms = [gbr, br, r, xgb, svr,l, enet]
names = ['Gradient Boosting', 'Bayesian Ridge', 'Ridge', 'XGB', 'SVR', 'Lasso','ElasticNet']
def tDMassess_regression():
    #fit the data
    for i in range(len(algorithms)):
        algorithms[i] = algorithms[i].fit(X_train,y_train)
    cv_rmse =[]
    rmse_train=[]
    rmse_test=[]
    for i in range(len(algorithms)):
        rmse_train.append(mean_squared_error(np.expm1(y_train), np.expm1(algorithms[i].predict(X_train))) **.5)
        rmse_test.append(mean_squared_error(np.expm1(y_test), np.expm1(algorithms[i].predict(X_test)))**.5)
        cv_rmse.append(cv_score(algorithms[i]))
    metrics = pd.DataFrame(columns =['RMSE_train', 'RMSE_test', 'cv_RMSE'], index=names)
    metrics['RMSE_train'] = rmse_train
    metrics['RMSE_test'] = rmse_test
    metrics['cv_RMSE'] = cv_rmse
    return metrics

In [ ]:
tDMassess_regression()

### Function to take the average of all the models
- not going to use SVR

In [ ]:
final_algs = [gbr, br, l,enet]
def average_of_models():
    final_pred=[]
    for i in range(len(final_algs)):
         final_pred.append(np.expm1(final_algs[i].predict(missing_price.drop('SalePrice', axis=1))))
    return (sum(final_pred)/len(final_algs))

In [ ]:
avg_preds = average_of_models()

## Best score on kaggle was .12425
- No label encoder
- and average of gbr, br, r, and xgb. 

### Now to Submit

In [ ]:
submission = pd.DataFrame()
submission['Id'] =sub_id
submission['SalePrice'] = avg_preds
submission.to_csv('final_sub_LE_la_enet.csv', index=False)